SAT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SAT version.

Necessary libraries:

In [1]:
!pip3 install z3-solver

Necessary imports:

In [2]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [3]:
# open the file in Instances folder
f = open("Instances/inst10.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 20
items: 215
load_size: [200, 180, 160, 185, 180, 180, 200, 160, 180, 185, 200, 160, 175, 180, 160, 200, 180, 175, 160, 200]
item_size: [3, 16, 8, 15, 12, 13, 8, 24, 14, 17, 8, 11, 8, 17, 19, 13, 2, 8, 21, 13, 17, 4, 22, 24, 11, 6, 19, 11, 23, 4, 5, 16, 2, 13, 11, 9, 18, 7, 24, 5, 2, 5, 5, 3, 13, 2, 10, 12, 24, 21, 18, 5, 15, 11, 7, 3, 17, 14, 14, 7, 12, 5, 11, 17, 3, 7, 13, 20, 13, 4, 3, 24, 23, 21, 23, 10, 21, 24, 9, 22, 3, 6, 11, 10, 15, 9, 20, 19, 19, 19, 20, 1, 15, 18, 2, 25, 20, 18, 8, 3, 2, 15, 21, 6, 24, 19, 23, 16, 16, 9, 1, 10, 14, 11, 4, 23, 23, 22, 11, 14, 12, 13, 11, 9, 6, 25, 21, 13, 23, 8, 7, 8, 9, 2, 24, 7, 23, 21, 12, 23, 18, 5, 15, 11, 8, 2, 6, 23, 9, 1, 12, 17, 2, 5, 1, 20, 19, 13, 10, 9, 14, 4, 20, 17, 19, 19, 16, 4, 1, 21, 9, 2, 24, 1, 21, 13, 15, 4, 10, 19, 1, 25, 3, 21, 7, 18, 3, 1, 2, 8, 10, 19, 17, 19, 24, 24, 6, 2, 7, 10, 13, 20, 1, 17, 17, 16, 9, 25, 1, 23, 10, 15, 25, 25, 16]
distance:
 [[  0  81  75 ...  40 158 103]
 [ 81   0  19 ...  59  82  83]

Other variables:

In [4]:
COURIERS = np.arange(m)
ITEMS = np.arange(n)
items_total = sum(item_size)
courier_per_item = [Int("c_%s" % (i)) for i in ITEMS]

In [5]:
courier_per_item

[c_0,
 c_1,
 c_2,
 c_3,
 c_4,
 c_5,
 c_6,
 c_7,
 c_8,
 c_9,
 c_10,
 c_11,
 c_12,
 c_13,
 c_14,
 c_15,
 c_16,
 c_17,
 c_18,
 c_19,
 c_20,
 c_21,
 c_22,
 c_23,
 c_24,
 c_25,
 c_26,
 c_27,
 c_28,
 c_29,
 c_30,
 c_31,
 c_32,
 c_33,
 c_34,
 c_35,
 c_36,
 c_37,
 c_38,
 c_39,
 c_40,
 c_41,
 c_42,
 c_43,
 c_44,
 c_45,
 c_46,
 c_47,
 c_48,
 c_49,
 c_50,
 c_51,
 c_52,
 c_53,
 c_54,
 c_55,
 c_56,
 c_57,
 c_58,
 c_59,
 c_60,
 c_61,
 c_62,
 c_63,
 c_64,
 c_65,
 c_66,
 c_67,
 c_68,
 c_69,
 c_70,
 c_71,
 c_72,
 c_73,
 c_74,
 c_75,
 c_76,
 c_77,
 c_78,
 c_79,
 c_80,
 c_81,
 c_82,
 c_83,
 c_84,
 c_85,
 c_86,
 c_87,
 c_88,
 c_89,
 c_90,
 c_91,
 c_92,
 c_93,
 c_94]

Formulation of the constraints:

In [5]:
constraints = []
# the constraint are the same as the CP model version 2
# number of items > number of couriers
constraints.append(n > m)

# Sum of item sizes cannot be bigger than total load size
constraints.append(items_total <= Sum([load_size[i] for i in COURIERS]))

# each item should have size > 0
for i in ITEMS:
    constraints.append(item_size[i] > 0)

# each courier should have load size > 0
for c in COURIERS:
    constraints.append(load_size[c] > 0)

# the couriers assigned should be between 0 and m-1
for i in ITEMS:
    constraints.append(And(courier_per_item[i] >= 0, courier_per_item[i] <= m-1))

# Each courier delivers at least one item
for c in COURIERS:
    constraints.append(Sum([If(courier_per_item[i] == c, 1, 0) for i in ITEMS]) >= 1)

# Each courier's items sizes must not excede his load size
for i in ITEMS:
    for c in COURIERS:
        constraints.append(Sum([If(courier_per_item[i] == c, item_size[i], 0) for i in ITEMS]) <= load_size[c])
    
constraints
# print number of constraints
print(len(constraints))

KeyboardInterrupt: 

In [7]:
constraints

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 And(c_0 >= 0, c_0 <= 19),
 And(c_1 >= 0, c_1 <= 19),
 And(c_2 >= 0, c_2 <= 19),
 And(c_3 >= 0, c_3 <= 19),
 And(c_4 >= 0, c_4 <= 19),
 And(c_5 >= 0, c_5 <= 19),
 And(c_6 >= 0, c_6 

Solving the model:

In [8]:
# add all the constraints to the solver
s = Solver()
s.add(constraints)
# solve
if s.check() == sat:
    m = s.model()
    # print the model in a readable format
    print("c_i = courier of item i")
    # make a list of the couriers assigned to each item
    for c in COURIERS:
        print("Courier", c, ":", [i for i in ITEMS if m[courier_per_item[i]] == c])
        
else:
    print("unsat")

c_i = courier of item i
Courier 0 : [58]
Courier 1 : [31, 34, 53, 89]
Courier 2 : [9, 11, 69]
Courier 3 : [38, 50, 76, 83]
Courier 4 : [77]
Courier 5 : [74]
Courier 6 : [81]
Courier 7 : [37, 66]
Courier 8 : [36, 63, 70]
Courier 9 : [62, 80]
Courier 10 : [52]
Courier 11 : [5, 7, 14, 19, 24, 27, 67]
Courier 12 : [39, 57, 72, 82]
Courier 13 : [21, 41, 46]
Courier 14 : [20, 28, 86]
Courier 15 : [16, 18, 49, 75]
Courier 16 : [12, 13, 17, 22, 23, 26, 29, 33, 54, 68, 73, 90, 93]
Courier 17 : [10, 32, 35, 43, 44, 56, 59, 60, 61, 65, 71, 79, 85, 87]
Courier 18 : [0, 1, 2, 3, 4, 6, 40, 45, 64, 78, 88, 91, 94]
Courier 19 : [8, 15, 25, 30, 42, 47, 48, 51, 55, 84, 92]
